In [ ]:
!pip install gensim nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.7/26.7 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 15.6 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.15.3
    Uninstalling scipy-1.15.3:
      Successfully uninstalled scipy-1.15.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
opencv-python-h

In [ ]:
from gensim.models import LdaModel
from gensim.corpora import Dictionary
from gensim.models.coherencemodel import CoherenceModel
import nltk,re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train=fetch_20newsgroups(subset='train',shuffle=True,random_state=42)
documents=newsgroups_train.data

In [ ]:
def clean_text(text):
  text=re.sub(r'\W+',' ',text.lower())
  tokens=[word for word in text.split() if word not in stop_words]
  return tokens

In [ ]:
#Tokenize all documents
tokenize_docs=[clean_text(doc) for doc in documents]

In [ ]:
dictionary=Dictionary(tokenize_docs)
dictionary.filter_extremes(no_below=10,no_above=0.9)
corpus=[dictionary.doc2bow(doc) for doc in tokenize_docs]

In [ ]:
lda_model=LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=8,
    random_state=42,
    alpha='auto',
    per_word_topics=True
    )

In [ ]:
print("\n TOP WORDS IN EACH TOPIC:")
for idx,topic in lda_model.show_topics(formatted=False):
  print(f"Topic{idx+1}:{'|'.join([word for word,_ in topic])}")


 TOP WORDS IN EACH TOPIC:
Topic1:god|x|one|would|people|edu|jesus|say|think|believe
Topic2:people|would|edu|one|government|com|key|think|writes|like
Topic3:ax|max|q|r|3|p|0d|g|7|g9v
Topic4:_|edu|1|x|com|graphics|2|file|mail|image
Topic5:edu|com|writes|article|posting|would|host|one|nntp|university
Topic6:0|1|2|3|4|5|7|6|8|25
Topic7:edu|com|1|one|use|drive|2|would|windows|system
Topic8:x|1|w|r|_|c|p|g|6|8


In [ ]:
coherence_model_lda=CoherenceModel(model=lda_model,texts=tokenize_docs,dictionary=dictionary,coherence='c_v')
coherence_lda=coherence_model_lda.get_coherence()
print(f"\n Coherence Score:{coherence_lda:.4f}")


 Coherence Score:0.6447


# Parameter What it Does Typical Values

In [ ]:
! pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 48.8 MB/s eta 0:00:00


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models

In [ ]:
vis_data=pyLDAvis.gensim_models.prepare(lda_model,corpus,dictionary,sort_topics=False)


In [ ]:
pyLDAvis.enable_notebook()
pyLDAvis.display(vis_data)